In [1]:
import yaml
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import KBinsDiscretizer
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import metrics, model_selection, ensemble


pd.set_option('display.max_columns', None)

C:\ProgramData\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
try:
    from CustomPipeline import *
except:
    import sys
    sys.path.insert(0,'/content/sample_data')
    from CustomPipeline import *
    print("ok")

In [3]:
try:
    train = pd.read_csv("./Datasets/train_anomaly.csv", delimiter=",", sep='.', index_col='id')
    test = pd.read_csv("./Datasets/test_anomaly.csv", delimiter=",", sep='.', index_col='id')
except:
    train = pd.read_csv("./sample_data/train_anomaly.csv", delimiter=",", sep='.')

In [4]:
with open("./Configs/params_model_linear.yml", 'r') as stream:
    try:
        params_linear = yaml.safe_load(stream)
        print(params_linear)
    except yaml.YAMLError as exc:
        print(exc)

{'alpha': 45.97949266795876}


In [5]:
RANDOM_STATE = 42

In [6]:
train["target"] = 100 * train["target"]

In [7]:
train, test_ensemble = train_test_split(train, test_size=0.1, random_state=RANDOM_STATE)
train_estimators, train_ensemble = train_test_split(train, test_size=0.2, random_state=RANDOM_STATE)

In [8]:
X = train_estimators.drop(["target"], axis=1)
y = train_estimators["target"]

In [3]:
!pip install dvc --user

  Using cached https://files.pythonhosted.org/packages/3a/f0/702721222ef6133b58cf1ee37484ecc6b79ac8175c2dd1a83d156ce181d8/dvc-2.6.4-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d3/cc/248dfb697bb98152c187ced584e505124ed952529d7f8e74b28f6f4d6f31/ruamel.yaml-0.17.16-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/9c/b9/84f5bd190a7dc234cea2ca1da02c8900ace982e0ad4ae494660585844f04/voluptuous-0.12.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/29/a465741a3d97ea3c17d21eaad4c64205428bde56742360876c4391f930d4/pathspec-0.8.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/e9/c4/ea9a6e34dd5c4c3049b0a86c49feb7f38c9f5e901ffb6eeb832e89dfa540/rich-10.9.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/aa/c8/1555defd0c6df2fde59f567dd812e5dcb2c843dac37dbbff2be66d6a6ca1/fsspec-2021.8.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/71/ce/35107e81e7

ERROR: spyder 3.3.6 requires pyqt5<5.13; python_version >= "3", which is not installed.
ERROR: spyder 3.3.6 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
!dvc --help

"dvc" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


# linear ridge

In [11]:
n_bins=63

X_lr = train_estimators.query("target > 600").drop(["target"], axis=1)
y_lr = train_estimators.query("target > 600")["target"]

lr = linear_model.Ridge(**params_linear)
model = LinearWrapper(lr, bins_linear=n_bins)
model.fit(X_lr, y_lr)

# save model

In [12]:
import pickle
import os

directory = './Models'
if not os.path.exists(directory):
    os.makedirs(directory)

# save
with open('./Models/model_ridge.pkl','wb') as f:
    pickle.dump(model,f)
